In [1]:
import os
import numpy as np
from pathlib import Path
import pyvista as pv

In [5]:
cyl_large = pv.Cylinder(center=[-21,-90,44], radius=15, direction=[0,1,0], height=20)

In [6]:
cyl_large.save("/home/max/Desktop/python_projects/SimJeb/SimJEB_metadata/cylinder_horizontal_large.stl")

In [8]:
mesh = pv.read("/home/max/Desktop/python_projects/SimJeb/SimJEB_surfmesh/0.obj")

In [11]:
clipped = cyl_large.clip_surface(mesh, True)

In [13]:
clipped

PolyData (0x7f5c13801cc0)
  N Cells:    0
  N Points:   0
  N Strips:   0
  X Bounds:   1.000e+299, -1.000e+299
  Y Bounds:   1.000e+299, -1.000e+299
  Z Bounds:   1.000e+299, -1.000e+299
  N Arrays:   3